In [18]:
import pandas as pd
import os
import cv2
import numpy as np
import json

import imutils
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

#Load index list
Year='1934'
Showa='9'


In [19]:
#Encoding Function
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)


In [20]:
### CLOVA FUNCTION ###
import requests
import uuid
import time
import json
import cv2
import base64

api_url = 'https://deelieyxuc.apigw.ntruss.com/custom/v1/1972/ebd01bcbf693d069817622e9839e20914143c7d0d8953eddee40e8b0af96c95b/general'
secret_key = 'S1NmVXpYZlJ0cGJ0ZEFRZXVlbkRkaHFReE9FcHNTQ0U='

def Clova(file_data):
    request_json = {
            'images': [
                {
                    'format': 'jpg',
                    'name': 'demo',
                    'data':base64.b64encode(file_data).decode()}],
            'requestId': str(uuid.uuid4()),
            'version': 'V2',
            'timestamp': int(round(time.time() * 1000)),
            'lang':'ja'
            }
    payload = json.dumps(request_json).encode("UTF-8")
    headers = {'X-OCR-SECRET': secret_key,
              'Content-Type': 'application/json'}
    response = requests.request("POST", api_url, headers=headers, data = payload)
    Json=json.loads(response.text)['images'][0]
    
    return Json    

In [21]:
#Function for Cell
def GetCell(cropped):
    #Code for Adding Grid
        ##Right page
        img = cropped.copy()
        hh, ww = img.shape[:2]

        #Identify grid location
        ## convert to grayscale
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        # threshold gray image
        thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]

        ## count number of non-zero pixels in each column and row. 
        countCol = np.count_nonzero(thresh, axis=0)
        countRow = np.count_nonzero(thresh, axis=1)

        ###############
        ## Column lines
        ###############
        ### This finds the height of the smallest peak
        peaksCol, _ = find_peaks(countCol, distance=10)
        ### threshold count at Thres
        count_thresh = countCol.copy()
        count_thresh[peaksCol] = 255
        count_thresh[count_thresh!=255] = 0
        count_thresh = count_thresh.astype(np.uint8)

        ### get contours
        contoursCol = cv2.findContours(count_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contoursCol = contoursCol[0] if len(contoursCol) == 2 else contoursCol[1]

        ### loop over contours and get bounding boxes and ycenter and draw horizontal line at ycenter
        result = cropped.copy()
        for cntr in contoursCol:
            x,y,w,h = cv2.boundingRect(cntr)
            ycenter = y
            cv2.line(result, (ycenter,0), (ycenter,hh), (255, 0, 0), 1)
        

        ###############
        ## Row lines
        ###############
        peaksRow, _ = find_peaks(countRow, distance=60)
        ### threshold count at Thres
        count_thresh = countRow.copy()
        count_thresh[peaksRow] = 255
        count_thresh[count_thresh!=255] = 0
        count_thresh = count_thresh.astype(np.uint8)

        ### get contours
        contoursRow = cv2.findContours(count_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contoursRow = contoursRow[0] if len(contoursRow) == 2 else contoursRow[1]

        ### loop over contours and get bounding boxes and ycenter and draw horizontal line at ycenter
        for cntr in contoursRow:
            x,y,w,h = cv2.boundingRect(cntr)
            ycenter = y+h//2
            cv2.line(result, (0,ycenter), (hh,ycenter), (255, 0, 0), 1)
                
        return(peaksRow,peaksCol)

In [22]:
def Extract(Position,ImageNumber):
    path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Office_Level\\"+Year+"\\"+Dept+"\\"+Office+"\\"+Position+"\\"
    stream = open(path+"Image"+str(ImageNumber)+'.png', "rb")
    bytes = bytearray(stream.read())
    numpyarray = np.asarray(bytes, dtype=np.uint8)
    img = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)

    HH,WW=img.shape[:2]
    
    dfA = pd.DataFrame(columns = ['Name', 'Wage'])
    dfT = pd.DataFrame(columns = ['Name', 'Wage'])
    dfB = pd.DataFrame(columns = ['Name', 'Wage'])
    
    if Position=="Admin":
        cropped=img[0:HH//2,0:WW]
        MiddleLineList=GetCell(cropped)[0]
        res = list(map(abs, [d-HH//4 for d in MiddleLineList.tolist()]))
        minpos = res.index(min(res))

        MiddleLine=MiddleLineList[minpos]
        ColumnLine=GetCell(cropped)[1]
        
        ##Top Chunk ##
        path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Temp\\"
        HH,WW=cropped.shape[:2]
        for Line in ColumnLine.tolist():
            if Line==ColumnLine.tolist()[0]:
                #Wage
                Image=cropped[0:MiddleLine,0:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Wage='NA'
                else:
                    Wage=''.join([d['inferText'] for d in Json['fields']])

                #Name
                Image=cropped[MiddleLine:HH,0:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Name='NA'
                else:
                    Name=''.join([d['inferText'] for d in Json['fields']])

                #Add to DF
                df2 = {'Name': Name, 'Wage': Wage}
                dfT = dfT.append(df2, ignore_index = True)
            else:
                #Wage
                Image=cropped[0:MiddleLine,Line-15:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Wage='NA'
                else:
                    Wage=''.join([d['inferText'] for d in Json['fields']])

                #Name
                Image=cropped[MiddleLine:HH,Line-15:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Name='NA'
                else:
                    Name=''.join([d['inferText'] for d in Json['fields']])

                #Add to DF
                df2 = {'Name': Name, 'Wage': Wage}
                dfT = dfT.append(df2, ignore_index = True)

        ##Bottom Chunk##
        cropped=img[HH//2:HH,0:WW]
        HH,WW=cropped.shape[:2]
        MiddleLineList=GetCell(cropped)[0]
        res = list(map(abs, [d-HH//4 for d in MiddleLineList.tolist()]))
        minpos = res.index(min(res))
        MiddleLine=MiddleLineList[minpos]
        ColumnLine=GetCell(cropped)[1]
        
        path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Temp\\"
        for Line in ColumnLine.tolist():
            if Line==ColumnLine.tolist()[0]:
                #Wage
                Image=cropped[0:MiddleLine,0:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Wage='NA'
                else:
                    Wage=''.join([d['inferText'] for d in Json['fields']])

                #Name
                Image=cropped[MiddleLine:HH,0:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Name='NA'
                else:
                    Name=''.join([d['inferText'] for d in Json['fields']])

                #Add to DF
                df2 = {'Name': Name, 'Wage': Wage}
                dfB = dfB.append(df2, ignore_index = True)
            else:
                #Wage
                Image=cropped[0:MiddleLine,Line-15:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Wage='NA'
                else:
                    Wage=''.join([d['inferText'] for d in Json['fields']])

                #Name
                Image=cropped[MiddleLine:HH,Line-15:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Name='NA'
                else:
                    Name=''.join([d['inferText'] for d in Json['fields']])

                #Add to DF
                #Add to DF
                df2 = {'Name': Name, 'Wage': Wage}
                dfB = dfB.append(df2, ignore_index = True)
        return pd.concat([dfT,dfB], ignore_index = True)
    
    else:
        cropped=img

        HH,WW=cropped.shape[:2]
        MiddleLineList=GetCell(cropped)[0]
        res = list(map(abs, [d-HH//2 for d in MiddleLineList.tolist()]))
        minpos = res.index(min(res))
        MiddleLine=MiddleLineList[minpos]
        ColumnLine=GetCell(cropped)[1].tolist()
        
        path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Temp\\"
        for Line in ColumnLine:
            if Line==ColumnLine[0]:
                #Wage
                Image=cropped[0:MiddleLine,0:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Wage='NA'
                else:
                    Wage=''.join([d['inferText'] for d in Json['fields']])

                #Name
                Image=cropped[MiddleLine:HH,0:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Name='NA'
                else:
                    Name=''.join([d['inferText'] for d in Json['fields']])

                #Add to DF
                df2 = {'Name': Name, 'Wage': Wage}
                dfA = dfA.append(df2, ignore_index = True)

            else:
                #Wage
                Image=cropped[0:MiddleLine,Line-15:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Wage='NA'
                else:
                    Wage=''.join([d['inferText'] for d in Json['fields']])

                #Name
                Image=cropped[MiddleLine:HH,Line-15:Line]
                cv2.imwrite(path+"Temp.jpg",Image)
                with open(path+"Temp.jpg",'rb') as f:
                    file_data = f.read()
                Json=Clova(file_data)
                if Json['inferResult']=='ERROR':
                    Name='NA'
                else:
                    Name=''.join([d['inferText'] for d in Json['fields']])


                #Add to DF
                df2 = {'Name': Name, 'Wage': Wage}
                dfA = dfA.append(df2, ignore_index = True)
        return(dfA)

In [23]:
#Load Data Frame
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\"+str(Year)+"\\DataFrame.json"
with open(path, 'r') as j:
     dta = json.loads(j.read())

df = pd.read_csv(r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Processed_Data/Index/S'+str(Showa)+'.csv')
df=df.drop(df.columns[0], axis=1)

In [14]:
#Test code| Version 2#
#Show Working office#
n=0

#Extract key info of office
Row  = df.iloc[n]
print(Row)
###Collect Location information###
Dept=Row["Dept"]
Office=Row["Office"]
PositionList=list(dta[str(Year)][Dept][Office]['Position'].keys())
print(PositionList)

for Position in PositionList:
    StrPage=int(dta[str(Year)][Dept][Office]['Position'][Position]['Page'])
    EndPage=int(dta[str(Year)][Dept][Office]['Position'][Position]['EndPage'])
    PageList=list(set([1,EndPage-StrPage+1]))
    print(Position)
    for ImageNumber in PageList:        
        print('Image Number is '+str(ImageNumber))
        #Download Image
        path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Office_Level\\"+Year+"\\"+Dept+"\\"+Office+"\\"+Position+"\\"
        try:
            stream = open(path+"Image"+str(ImageNumber)+'.png', "rb")
            bytes = bytearray(stream.read())
            numpyarray = np.asarray(bytes, dtype=np.uint8)
            img = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)
        except:
            print('Could not find image')
            continue

        HH,WW=img.shape[:2]

        DF=pd.DataFrame(columns = ['Name', 'Wage'])
        if Position=='Admin':
            croppedTop=img[0:HH//2,0:WW]
            cv2.imshow("Sample",croppedTop)
            cv2.waitKey(0)

            croppedBtm=img[HH//2:HH,0:WW]
            cv2.imshow("Sample",croppedBtm)
            cv2.waitKey(0)
            
            dta[str(Year)][Dept][Office]['Position'][Position]['Data']=Extract(Position,ImageNumber)

        else:
            cropped=img
            cv2.imshow("Sample",cropped)
            cv2.waitKey(0)
            
            dta[str(Year)][Dept][Office]['Position'][Position]['Data']=Extract(Position,ImageNumber)


Office      秘書課
Dept      Admin
Year          9
Page          2
Name: 0, dtype: object
['Leader', 'Manager', 'Admin']
Leader
Image Number is 1
Could not find image
Manager
Image Number is 1
Could not find image
Admin
Image Number is 1
Could not find image


In [24]:
#Test code| Version 2#
#Show Working office#
for n in range(1,len(df)):
    #Extract key info of office
    Row  = df.iloc[n]
    print(Row)
    ###Collect Location information###
    Dept=Row["Dept"]
    Office=Row["Office"]
    try:
        PositionList=list(dta[str(Year)][Dept][Office]['Position'].keys())
    except:
        continue
    print(PositionList)

    for Position in PositionList:
        StrPage=int(dta[str(Year)][Dept][Office]['Position'][Position]['Page'])
        EndPage=int(dta[str(Year)][Dept][Office]['Position'][Position]['EndPage'])
        PageList=list(set([1,EndPage-StrPage+1]))
        print(Position)
        for ImageNumber in PageList:        
            #Download Image
            path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Office_Level\\"+Year+"\\"+Dept+"\\"+Office+"\\"+Position+"\\"
            try:
                stream = open(path+"Image"+str(ImageNumber)+'.png', "rb")
                bytes = bytearray(stream.read())
                numpyarray = np.asarray(bytes, dtype=np.uint8)
                img = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)
            except:
                print('Could not find image')
                continue

            HH,WW=img.shape[:2]

            DF=pd.DataFrame(columns = ['Name', 'Wage'])
            if Position=='Admin':
                croppedTop=img[0:HH//2,0:WW]
                croppedBtm=img[HH//2:HH,0:WW]
                try:
                    dta[str(Year)][Dept][Office]['Position'][Position]['Data']=Extract(Position,ImageNumber)
                except:
                    continue

            else:
                cropped=img
                try:
                    dta[str(Year)][Dept][Office]['Position'][Position]['Data']=Extract(Position,ImageNumber)
                except:
                    continue


Office      職員課
Dept      Admin
Year          9
Page          2
Name: 1, dtype: object
['Leader', 'Manager', 'Admin']
Leader


C:\Temp\ipykernel_26788\1440442662.py:183: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)


Manager


C:\Temp\ipykernel_26788\1440442662.py:183: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)
C:\Temp\ipykernel_26788\1440442662.py:211: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)
C:\Temp\ipykernel_26788\1440442662.py:211: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)
C:\Temp\ipykernel_26788\1440442662.py:211: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)
C:\Temp\ipykernel_26788\1440442662.py:211: FutureWarning: The frame.append method is deprecated and will be removed 

Could not find image
Admin
Could not find image
Office      文書課
Dept      Admin
Year          9
Page          3
Name: 2, dtype: object
['Manager', 'Leader', 'Admin']
Manager


...

C:\Temp\ipykernel_26788\1440442662.py:183: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)
C:\Temp\ipykernel_26788\1440442662.py:211: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)
C:\Temp\ipykernel_26788\1440442662.py:211: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)
C:\Temp\ipykernel_26788\1440442662.py:211: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfA = dfA.append(df2, ignore_index = True)
C:\Temp\ipykernel_26788\1440442662.py:211: FutureWarning: The frame.append method is deprecated and will be removed 

Admin


C:\Temp\ipykernel_26788\1440442662.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfT = dfT.append(df2, ignore_index = True)
C:\Temp\ipykernel_26788\1440442662.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfT = dfT.append(df2, ignore_index = True)
C:\Temp\ipykernel_26788\1440442662.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfT = dfT.append(df2, ignore_index = True)
C:\Temp\ipykernel_26788\1440442662.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfT = dfT.append(df2, ignore_index = True)
C:\Temp\ipykernel_26788\1440442662.py:78: FutureWarning: The frame.append method is deprecated and will be removed from 

Could not find image
Engineer2
Could not find image
Office    衛生課
Dept      保健局
Year        9
Page       51
Name: 28, dtype: object
Office    清掃課
Dept      保健局
Year        9
Page       54
Name: 29, dtype: object
Office    公園課
Dept      保健局
Year        9
Page       62
Name: 30, dtype: object
Office    庶務課
Dept      水道局
Year        9
Page       64
Name: 31, dtype: object
Office    会計課
Dept      水道局
Year        9
Page       66
Name: 32, dtype: object
Office    業務課
Dept      水道局
Year        9
Page       66
Name: 33, dtype: object
Office    給水課
Dept      水道局
Year        9
Page       72
Name: 34, dtype: object
Office    拡張課
Dept      水道局
Year        9
Page       74
Name: 35, dtype: object
Office    庶務課
Dept      土木局
Year        9
Page       77
Name: 36, dtype: object
Office    道路管理課
Dept        土木局
Year          9
Page         79
Name: 37, dtype: object
Office    道路建設課
Dept        土木局
Year          9
Page         81
Name: 38, dtype: object
Office    河川課
Dept      土木局
Year        9
Page      

C:\Temp\ipykernel_26788\1440442662.py:143: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfB = dfB.append(df2, ignore_index = True)


In [42]:
Frame=pd.DataFrame(columns=['Dept', 'Office', 'Position','Name','Wage'])
FailedList=[]
for n in range(0,len(df)):
    Row=df.iloc[n]
    Dept=Row['Dept']
    Office=Row['Office']
    try:
        PosiList=list(dta[Year][Dept][Office]['Position'].keys())
    except:
        FailedList.append(list((Dept,Office)))
        continue
        
    for Posi in PosiList:
        try:
            NewFrame=dta[Year][Dept][Office]['Position'][Posi]['Data']
            NewFrame['Dept']=Dept
            NewFrame['Office']=Office
            NewFrame['Position']=Position
            Frame=pd.concat([Frame,NewFrame])
        except:
            FailedList.append(list((Dept,Office)))
            continue    
Frame.to_csv('C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Processed_Data/'+Year+'/Final'+Year+'.csv')

In [40]:
FailRate=len(FailedList)/len(df)
if len(FailedList)/len(df)<0.2:
    print('Fantastic!! Success Rate is '+str(1-(len(FailedList)/len(df))))
else:
    print('残念...Success Rate is '+str(1-(len(FailedList)/len(df))))
DF=pd.read_csv('C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\Records.csv')
DF.loc[int(Year)-1934, 'Position'] = 1-FailRate
DF.to_csv('C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\Records.csv')

残念...Success Rate is -0.016666666666666607


In [43]:
display(DF)

,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Year,Page,Office,Unit,Position,Name
0,0,0,0,0,0,0,0,1934,0.985714,0.900000,NaN,-0.016667,Nino
1,1,1,1,1,1,1,1,1935,NaN,0.859375,NaN,NaN,Nino
2,2,2,2,2,2,2,2,1936,NaN,NaN,NaN,NaN,Yuya
3,3,3,3,3,3,3,3,1937,NaN,NaN,NaN,NaN,Yuya
4,4,4,4,4,4,4,4,1938,NaN,NaN,0.464286,NaN,Nino
5,5,5,5,5,5,5,5,1939,NaN,NaN,NaN,NaN,Yuya
6,6,6,6,6,6,6,6,1940,NaN,NaN,NaN,NaN,Nino
7,7,7,7,7,7,7,7,1941,NaN,NaN,NaN,NaN,Yuya
8,8,8,8,8,8,8,8,1942,NaN,NaN,NaN,NaN,NaN
9,9,9,9,9,9,9,9,1943,NaN,NaN,NaN,NaN,NaN
